In [ ]:
import numpy as np
import os
from typing import List, Tuple,Dict
import json
from evaluate_utils import MAE, RMSE
from collections import defaultdict

In [ ]:
DATASET_PATH = "../../dataset"
RESULT_ROOT_PATH = "results"
# MODEL_NAME = "lstm"
# MODEL_NAME = "nlinear"
# MODEL_NAME = "segrnn"
MODEL_NAME="patchtst"


EVAL_ROOT_PATH = "eval"

eval_result_dict = defaultdict(float)

In [ ]:
def get_data_file_list(dataset_path: str) -> List[str]:
    return os.listdir(dataset_path)

In [ ]:
def read_result(file_name: str) -> np.ndarray:
    dir = os.path.join(RESULT_ROOT_PATH, MODEL_NAME)
    result = None
    with open(os.path.join(dir, file_name), "r") as f:
        result = json.load(f)
    result = np.array(result)
    return result

In [ ]:
def get_gt_and_pd(workload_name: str, name_split: str, is_scaled: bool) -> Tuple[np.ndarray, np.ndarray]:
    gt_file_name = None
    if is_scaled:
        gt_file_name = workload_name+"_gt_"+name_split+"_"+"scaled.json"
    else:
        gt_file_name = workload_name+"_gt_"+name_split+"_"+"original.json"

    pd_file_name = None
    if is_scaled:
        pd_file_name = workload_name+"_pd_"+name_split+"_"+"scaled.json"
    else:
        pd_file_name = workload_name+"_pd_"+name_split+"_"+"original.json"

    gt = read_result(gt_file_name)
    pd = read_result(pd_file_name)
    
    gt_0=gt[:,:,0,:]
    pd_0=pd[:,:,0,:]
    return gt_0, pd_0

In [ ]:
def save_to_file(result_dict: Dict, file_name: str):
    dir_name = os.path.join(EVAL_ROOT_PATH, MODEL_NAME)
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    with open(os.path.join(dir_name, file_name)+".json", "w") as f:
        json.dump(result_dict, f, indent=4)

In [ ]:
data_file_list = get_data_file_list(DATASET_PATH)
is_scaled = False
for name_split in ["train", "test"]:
    eval_result_dict.clear()
    for file_name in data_file_list:
        workload_name = file_name.split(".")[0]
        gt, pd = get_gt_and_pd(workload_name, name_split, is_scaled)
        mae = MAE(gt[:, 0], pd[:, 0])
        eval_result_dict[workload_name] = mae
    save_file_name = name_split
    if is_scaled:
        save_file_name += "_scaled.json"
    else:
        save_file_name += "_original.json"
    save_to_file(eval_result_dict, save_file_name)